## Data Architecture Overview

### Data Pipelines

![updated_aeon_pipelines.png](updated_aeon_pipelines.png)

### Data Storage

![data_storage.png](data_storage.png)

### Code Repo Structure

"prod", "main", and feature branches

![code_repos_structure.jpg](code_repos_structure.jpg)

## 'aeon_mecha' Updates

### Updated aeon_mecha repo structure

![aeon_mecha_reorg.png](aeon_mecha_reorg.png)

### Set "schemas" for different experiment types

Allows for general use of low-level api across various experiments

![reorg_schemas.png](reorg_schemas.png)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import aeon.io.api as api
from aeon.schema.dataset import exp02, exp01
from aeon.analysis.utils import visits

In [10]:
root_exp01 = '/ceph/aeon/aeon/data/raw/AEON/experiment0.1'
root_exp02 = '/ceph/aeon/aeon/data/raw/AEON2/experiment0.2'
# find all "session" Start and End events
exp01_events = api.load(root_exp01, exp01.SessionData)
exp02_events = api.load(root_exp02, exp02.ExperimentalMetadata.SubjectState)
print(exp01_events.head())
print(exp02_events.head())

                                        id  weight  event
time                                                     
2021-06-03 08:37:52.918399811  BAA-1099790    25.2  Start
2021-06-03 13:18:41.202020168  BAA-1099790    26.9    End
2021-06-03 13:49:49.323520184  BAA-1099794    21.9  Start
2021-06-03 18:01:51.744060040  BAA-1099794    26.2    End
2021-06-04 07:46:35.178850174  BAA-1099790    24.9  Start
                               weight   event     id
time                                                
2022-02-17 14:07:33.641499996   200.0  Remain  Dario
2022-02-17 14:08:09.863490105   200.0  Remain  Dario
2022-02-17 14:09:26.629499912   200.0  Remain  Dario
2022-02-17 14:10:20.787489891   200.0  Remain  Dario
2022-02-17 14:16:09.889500141   200.0  Remain  Dario


In [11]:
exp01_sessions = exp01_events[exp01_events.id.str.startswith('BAA')]  # take only proper sessions
if len(exp01_sessions) % 2 != 0:                                      # if number of sessions don't pair up
    exp01_sessions = exp01_sessions.drop(exp01_sessions.index[-1])    # drop last session (might be ongoing)
print(exp01_sessions.head())
exp01_sessions = visits(exp01_sessions, onset='Start', offset='End')  # compute session duration


exp02_sessions = exp02_events[exp02_events.id.str.startswith('BAA')]              
if len(exp02_sessions) % 2 != 0:                                      
    exp02_sessions = exp02_sessions.drop(exp02_sessions.index[-1])    
print(exp02_sessions.head())
exp02_sessions = visits(exp02_sessions)

print(exp01_sessions.head())
print(exp02_sessions.head())

                                        id  weight  event
time                                                     
2021-06-03 08:37:52.918399811  BAA-1099790    25.2  Start
2021-06-03 13:18:41.202020168  BAA-1099790    26.9    End
2021-06-03 13:49:49.323520184  BAA-1099794    21.9  Start
2021-06-03 18:01:51.744060040  BAA-1099794    26.2    End
2021-06-04 07:46:35.178850174  BAA-1099790    24.9  Start
                               weight   event           id
time                                                      
2022-02-23 14:21:25.550300121    26.4   Enter  BAA-1100700
2022-02-23 14:32:56.998499870    26.4  Remain  BAA-1100700
2022-02-23 17:03:02.676479816    26.4  Remain  BAA-1100700
2022-02-23 17:18:02.294499874    26.4  Remain  BAA-1100700
2022-02-23 17:26:35.470499992    26.4  Remain  BAA-1100700
            id  weight_start weight_end                         start  \
0  BAA-1099790          25.2       26.9 2021-06-03 08:37:52.918399811   
1  BAA-1099794          21.9       

In [12]:
pos_exp01_ses1 = api.load(root_exp01, exp01.FrameTop.Position, start=exp01_sessions.iloc[1].start, end=exp01_sessions.iloc[1].end)
pos_exp02_visit2 = api.load(root_exp02, exp02.CameraTop.Position, start=exp02_sessions.iloc[2].enter, end=exp02_sessions.iloc[2].exit)

print(pos_exp01_ses1.head())
print(pos_exp02_visit2.head())

                                        x           y     angle      major  \
time                                                                         
2021-06-03 13:49:49.331552029  321.766296  769.737549  1.053047  36.865139   
2021-06-03 13:49:49.351520061  323.031616  772.197937  1.030587  36.683720   
2021-06-03 13:49:49.371551991  324.259247  774.437317  1.009591  35.807312   
2021-06-03 13:49:49.391520023  325.546692  776.410095  1.027776  35.810875   
2021-06-03 13:49:49.411551952  327.086639  778.393127  1.048764  36.244995   

                                   minor   area  id  
time                                                 
2021-06-03 13:49:49.331552029  13.182050  498.5 NaN  
2021-06-03 13:49:49.351520061  13.237064  501.0 NaN  
2021-06-03 13:49:49.371551991  13.206387  490.5 NaN  
2021-06-03 13:49:49.391520023  12.980263  482.0 NaN  
2021-06-03 13:49:49.411551952  13.075988  490.5 NaN  
                                x   y  angle  major  minor  area  id
time  

## Datajoint Updates

In [4]:
import datajoint as dj
dj.config['database.host'] = 'aeon-db2'
dj.config['display.limit'] = 5  # rows per displayed table
dj.conn()
from aeon.dj_pipeline.analysis.in_arena import *
from aeon.dj_pipeline.acquisition import WeightMeasurement, ExperimentLog, ExperimentWeightScale, Chunk, Experiment, _device_schema_mapping
from aeon.dj_pipeline.report import *
from aeon.io import api as io_api
from aeon.dj_pipeline.utils.plotting import *

[2022-09-09 11:54:25,649][INFO]: Connecting jbhagat@aeon-db2:3306
[2022-09-09 11:54:25,664][INFO]: Connected jbhagat@aeon-db2:3306


In [21]:
# Integration with new api, handling of multi-experiment types 
# (e.g. session single-animal, continous single-animal, session multi-animal)

dj.Diagram(Experiment) + 2
print(Experiment.heading)
print(Experiment.head)
print(Visit.heading)
print(Visit.head)

# 
experiment_name      : char(12)                     # e.g exp0-a
---
experiment_start_time : datetime(6)                  # datetime of the start of this experiment
experiment_description : varchar(1000)                # 
arena_name           : varchar(32)                  # unique name of the arena (e.g. circular_2m)
lab                  : varchar(24)                  # Abbreviated lab name
location             : varchar(32)                  # 
experiment_type="foraging" : varchar(32)                  # 

<bound method QueryExpression.head of *experiment_name    experiment_start_time     experiment_description     arena_name     lab     location     experiment_type    
+-----------------+ +-----------------------+ +------------------------+ +------------+ +-----+ +----------+ +-----------------+
exp0.1-r0           2021-06-03 07:00:00       experiment 0.1             circle-2m      SWC     room-0       foraging           
exp0.2-r0           2022-02-22 09:00:00       experiment 0.2

## Fall / Winter Plans

- Extending analysis of multianimal experiments

	- Online multianimal tracking and pose estimation with ma-DLC and SLEAP

	- Adding 1) multianimal body part position and 2) behaviour classification data to database

- Brainstorming ephys pipelines

	- Preprocessing of ephys data (e.g. spikesorting and LFP band preprocessing)

	- Structuring ephys data in the database

- Adding a video streamer/viewer to web GUI

- Auto-ingestion of any new device data into the database

- Continuous, online device qc

- Updating experiment metadata files

- Datajoint + pyrat integration